## <center> <font color='purple'> Dynamic Ensemble Machine Learning Models (Mix Pool)</font></center> 
#### <center>Firuz Juraev (Sungkyunkwan Unniversity)</center>

### <font color='green'> Libraries 

#### <font color='blue'> Basic Libraries 

In [1]:
import pandas as pd
import numpy as np 
from numpy import mean
from numpy import std
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### <font color='blue'> Single ML Models Libraries 

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#### <font color='blue'> Static ML Models Libraries 

In [3]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

#### <font color='blue'> DES Libraries 

In [4]:
from deslib.des import DESP
from deslib.des import KNORAE
from deslib.des import KNORAU
from deslib.des import METADES
from deslib.des import DESKNN # new 
from deslib.des import KNOP # new 

#### <font color='blue'> DCS Libraries 

In [5]:
from deslib.dcs import MCB

#### <font color='blue'> Processing Libraries 

In [6]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score, 
                             f1_score,
                             roc_auc_score, 
                             auc)
from sklearn.metrics import roc_curve, roc_auc_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### <font color='green'> Load Dataset 

In [7]:
train_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_train_data_4.csv")
test_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_test_data_4.csv")

X_train = train_dataset_upsampled.drop(["DEAD"], axis=1) 
y_train = train_dataset_upsampled["DEAD"]

X_test = test_dataset_upsampled.drop(["DEAD"], axis=1) 
y_test = test_dataset_upsampled["DEAD"]

In [8]:
columns = X_train.columns.to_list()

### <font color='green'> Data Normalization

In [9]:
Min_max_scaler = MinMaxScaler().fit(X_train)

## Scaling 
X_train_mm_scaled = Min_max_scaler.transform(X_train)
X_test_mm_scaled = Min_max_scaler.transform(X_test)

## Numpy Array to DataFrame 
df_train_mm_scaled = pd.DataFrame(X_train_mm_scaled, columns = columns)
df_test_mm_scaled = pd.DataFrame(X_test_mm_scaled, columns = columns)

### <font color='green'> Feature Selection

In [10]:
tain_mm_scaled_df = df_train_mm_scaled[:]
tain_mm_scaled_df["DEAD"] = y_train 

test_mm_scaled_df = df_test_mm_scaled[:] 
test_mm_scaled_df["DEAD"] = y_test


importances = mutual_info_classif(df_train_mm_scaled, y_train)
feat_importance = pd.Series(importances, tain_mm_scaled_df.columns[0:len(tain_mm_scaled_df.columns)-1])
    
feat_importance = feat_importance.sort_values(ascending=False)
    
selected_features = feat_importance[:30]
selected_features_list_mm_scaled = selected_features.index.to_list()


tain_mm_scaled_df[selected_features_list_mm_scaled].head(2)

,temperature_mean,respRate_std,respRate_var,skinTemperature_var,skinTemperature_std,heartRate_std,heartRate_var,bpCuffMean_var,sao2_std,sao2_var,...,BIRTH_WEIGHT,bpCuffSystolic_mean,bpCuffDiastolic_mean,temperature_var,sao2_mean,temperature_std,glucometer_mean,bpCuffMean_mean,PLATELET,D10W_MEAN
0,0.775318,0.422245,0.178291,0.017977,0.134078,0.22911,0.057937,0.053294,0.097817,0.009568,...,0.264267,0.602649,0.457143,0.015695,0.881455,0.125281,0.312593,0.310696,0.281330,0.151138
1,0.709668,0.583964,0.341014,0.007512,0.086672,0.28258,0.086102,0.020317,0.080938,0.006551,...,0.437819,0.644907,0.479330,0.003770,0.926056,0.061399,0.272551,0.351928,0.392157,0.196784


In [11]:
X_train_mm = df_train_mm_scaled[selected_features_list_mm_scaled][:]
X_test_mm = df_test_mm_scaled[selected_features_list_mm_scaled][:]

### <font color='purple'> Hold-out Test (With Mix ML) - (+FS, +HO)

In [12]:
def get_individual_result(model, dsel_x, dsel_y, test_x, test_y): 
    model.fit(dsel_x, dsel_y)
    y_preds = model.predict(test_x) 
    yproba = model.predict_proba(test_x)[::,1] 
    
    acc = accuracy_score(test_y, y_preds)
    prec = precision_score(test_y, y_preds)
    rec = recall_score(test_y, y_preds)
    f1 = f1_score(test_y, y_preds)
    fpr, tpr, _ = roc_curve(test_y,  yproba) 
    auc = roc_auc_score(test_y, yproba)
    
    return {"acc": acc, "prec": prec, "rec": rec, "f1":f1, "fpr": fpr, "tpr":tpr, "auc": auc}
    

In [13]:
def hold_out_mix_ML(): 
    rng = np.random.RandomState(42) 
    X_train, X_dsel, y_train_en, y_dsel = train_test_split(X_train_mm, y_train, test_size=0.40, random_state=rng)
    model_dt1 = DecisionTreeClassifier(criterion='entropy', max_depth=3)
    model_dt2 = DecisionTreeClassifier(criterion='entropy', max_depth=3)
    model_dt3 = DecisionTreeClassifier(criterion='entropy', max_depth=3)
    model_dt4 = DecisionTreeClassifier(criterion='entropy', max_depth=3)
    model_dt5 = DecisionTreeClassifier(criterion='entropy', max_depth=3)
    voting_classifiers = [("dt1", model_dt1),
                          ("dt2", model_dt2),
                          ("dt3", model_dt3),
                          ("dt4", model_dt4), 
                          ("dt5", model_dt5)]
    
    model_svc = SVC(kernel='linear', C=0.007, gamma=0.2, degree=3, probability=True, class_weight='balanced')
    model_dt = DecisionTreeClassifier(criterion='entropy', max_depth=3) # depth was 3  
    model_lr = LogisticRegression(penalty='l2', C=0.002)
    model_ml_perceptron = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    model_nb = GaussianNB(var_smoothing=0.1)
    
    model_rf  = RandomForestClassifier(criterion='gini', n_estimators=100, max_depth=3)
    model_cat = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    model_lgb = LGBMClassifier(max_depth=1, n_estimators=150, objective="binary")
    model_voting = VotingClassifier(estimators = voting_classifiers, voting='soft') 
    
    model_svc.fit(X_train, y_train_en)
    model_dt.fit(X_train, y_train_en)
    model_lr.fit(X_train, y_train_en)
    model_ml_perceptron.fit(X_train, y_train_en)
    model_nb.fit(X_train, y_train_en)
    
    model_rf.fit(X_train, y_train_en) 
    model_cat.fit(X_train, y_train_en) 
    model_lgb.fit(X_train, y_train_en)
    model_voting.fit(X_train, y_train_en)
    
    classifiers_names = ["MLP", "Decision Tree", "Logistic Regression", "SVC", "NB", "Random Forest", 
                         "CatBoost", "LGBM", "MajorityVoting"] # "Decision Tree",
    # "AdaBoost" , "LGBM"
    pool_classifiers = [model_ml_perceptron, 
                        model_dt,
                        model_lr, 
                        model_svc, 
                        model_nb, 
                        model_rf, 
                        model_cat,
                        model_lgb,
                        model_voting
                        ] 
    c_acc_list = [] 
    
    for cls in pool_classifiers:
        y_preds_c = cls.predict(X_test_mm) 
        c_acc_list.append(accuracy_score(y_test, y_preds_c))           
        
        
    classifiers_results =  {'name': classifiers_names, 
                            'accuracy': c_acc_list}
    
    clsDF = pd.DataFrame.from_dict(classifiers_results)
        
    # DES STARTS
    
    knorau = KNORAU(pool_classifiers)
    kne = KNORAE(pool_classifiers)  
    metades = METADES(pool_classifiers)
    desknn = DESKNN(pool_classifiers)
    mcb = MCB(pool_classifiers)
    desp = DESP(pool_classifiers)
    knop = KNOP(pool_classifiers)

    fire_knorau = KNORAU(pool_classifiers, DFP=True, k=7) 
    fire_kne = KNORAE(pool_classifiers, DFP=True, k=9)
    fire_metades = METADES(pool_classifiers, DFP=True, k=9) 
    fire_desknn = DESKNN(pool_classifiers, DFP=True, k=9)
    fire_mcb = MCB(pool_classifiers, DFP=True, k=7) # 7 was 96%
    fire_desp = DESP(pool_classifiers, DFP=True, k=9)
    fire_knop = KNOP(pool_classifiers, DFP=True, k=15)
    
    ensemble_classifiers = [fire_knorau, knorau, fire_kne, kne, fire_metades, metades, fire_desknn, desknn, 
                            fire_mcb, mcb, fire_desp, desp, fire_knop, knop]
    
    ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                      "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]
    
    acc_list = [] 
    precision_list = [] 
    recall_list = []
    f1_lists = [] 
    auc_list = [] 
    fpr_list = [] 
    tpr_list = []
    
    for e_cls in ensemble_classifiers:
        results_dct = get_individual_result(e_cls, X_dsel, y_dsel, X_test_mm, y_test)
        acc_list.append(results_dct["acc"])
        precision_list.append(results_dct["prec"])
        recall_list.append(results_dct["rec"]) 
        f1_lists.append(results_dct["f1"]) 
        auc_list.append(results_dct["auc"])
        fpr_list.append(results_dct["fpr"])
        tpr_list.append(results_dct["tpr"])
        

    results =  {'name': ensemble_names, 
                'accuracy': acc_list, 
                'precision': precision_list, 
                'recall': recall_list, 'f1': f1_lists, 
                'auc': auc_list, 
                'tpr': tpr_list, 'fpr': fpr_list}
    
    df = pd.DataFrame.from_dict(results)
    
    return df, clsDF

In [14]:
results_data = []
classifier_results_data = []
for i in range(0, 15):
    result, cls_results = hold_out_mix_ML()
    results_data.append(result)
    classifier_results_data.append(cls_results)
    

fireResultsDF = pd.concat(results_data)
classifiersResultsDF = pd.concat(classifier_results_data)

Iteration 1, loss = 0.65725992
Iteration 2, loss = 0.54290633
Iteration 3, loss = 0.45412108
Iteration 4, loss = 0.39418061
Iteration 5, loss = 0.36012913
Iteration 6, loss = 0.33657826
Iteration 7, loss = 0.32140213
Iteration 8, loss = 0.30845671
Iteration 9, loss = 0.29507320
Iteration 10, loss = 0.28724066
Iteration 11, loss = 0.28159370
Iteration 1, loss = 0.64464548
Iteration 2, loss = 0.53142127
Iteration 3, loss = 0.44548697
Iteration 4, loss = 0.39387722
Iteration 5, loss = 0.36123517
Iteration 6, loss = 0.33939984
Iteration 7, loss = 0.32395209
Iteration 8, loss = 0.31053943
Iteration 9, loss = 0.29903933
Iteration 10, loss = 0.29111611
Iteration 11, loss = 0.28104000
Iteration 1, loss = 0.64651586
Iteration 2, loss = 0.54006920
Iteration 3, loss = 0.45743181
Iteration 4, loss = 0.40262038
Iteration 5, loss = 0.36978674
Iteration 6, loss = 0.34473797
Iteration 7, loss = 0.32793946
Iteration 8, loss = 0.31469228
Iteration 9, loss = 0.30358861
Iteration 10, loss = 0.29213466
Ite

Iteration 1, loss = 0.66440524
Iteration 2, loss = 0.55570646
Iteration 3, loss = 0.46961350
Iteration 4, loss = 0.41138487
Iteration 5, loss = 0.37717879
Iteration 6, loss = 0.35226548
Iteration 7, loss = 0.33484702
Iteration 8, loss = 0.32228446
Iteration 9, loss = 0.31522031
Iteration 10, loss = 0.30187492
Iteration 11, loss = 0.29411281
Iteration 1, loss = 0.61152106
Iteration 2, loss = 0.50186564
Iteration 3, loss = 0.42003485
Iteration 4, loss = 0.37239793
Iteration 5, loss = 0.34483729
Iteration 6, loss = 0.32429187
Iteration 7, loss = 0.31068264
Iteration 8, loss = 0.29710431
Iteration 9, loss = 0.28835815
Iteration 10, loss = 0.28080063
Iteration 11, loss = 0.27436636
Iteration 1, loss = 0.63998370
Iteration 2, loss = 0.52533543
Iteration 3, loss = 0.43824370
Iteration 4, loss = 0.38686692
Iteration 5, loss = 0.35614834
Iteration 6, loss = 0.33916336
Iteration 7, loss = 0.32514375
Iteration 8, loss = 0.31036388
Iteration 9, loss = 0.29757565
Iteration 10, loss = 0.29321425
Ite

In [15]:
# fireResultsDF.to_csv("Results/des_mix_pool_7_cls_results_extend.csv", index=False)

In [16]:
ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                  "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]

accuracy = [] 
accuracy_std =  [] 
precision = [] 
precision_std = [] 
recall = [] 
recall_std = [] 
f1_score = [] 
f1_std = []
auc_score = []
auc_std = []


for n in ensemble_names:
    accuracy.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.mean(), 3))
    accuracy_std.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.std(), 3))
    precision.append(round(fireResultsDF[fireResultsDF.name == n].precision.mean(), 3))
    precision_std.append(round(fireResultsDF[fireResultsDF.name == n].precision.std(), 3))
    recall.append(round(fireResultsDF[fireResultsDF.name == n].recall.mean(), 3))
    recall_std.append(round(fireResultsDF[fireResultsDF.name == n].recall.std(), 3))
    f1_score.append(round(fireResultsDF[fireResultsDF.name == n].f1.mean(), 3))
    f1_std.append(round(fireResultsDF[fireResultsDF.name == n].f1.std(), 3))
    auc_score.append(round(fireResultsDF[fireResultsDF.name == n].auc.mean(), 3))
    auc_std.append(round(fireResultsDF[fireResultsDF.name == n].auc.std(), 3))
    
final_results = {"method": ensemble_names, 
                     "accuracy": accuracy, 
                     "accuracy_std": accuracy_std,
                     "precision": precision, 
                     "precision_std": precision_std,
                     "recall": recall, 
                     "recall_std": recall_std,
                     "f1_score": f1_score, 
                     "f1_std": f1_std, 
                     "auc": auc_score, 
                     "auc_std": auc_std}

finalResultsDF = pd.DataFrame.from_dict(final_results)

In [17]:
finalResultsDF

,method,accuracy,accuracy_std,precision,precision_std,recall,recall_std,f1_score,f1_std,auc,auc_std
0,FIRE-KNORA-U,0.973,0.001,0.949,0.002,1.000,0.000,0.974,0.001,0.992,0.000
1,KNORA-U,0.972,0.002,0.948,0.003,1.000,0.000,0.973,0.001,0.992,0.000
2,FIRE-KNORA-E,0.942,0.001,0.969,0.002,0.914,0.000,0.941,0.001,0.909,0.000
3,KNORA-E,0.944,0.001,0.973,0.002,0.914,0.000,0.943,0.001,0.909,0.000
4,FIRE-METADES,0.959,0.025,0.968,0.002,0.950,0.051,0.958,0.026,0.979,0.005
5,METADES,0.950,0.020,0.972,0.002,0.927,0.041,0.948,0.021,0.974,0.018
6,FIRE-DESKNN,0.971,0.015,0.956,0.002,0.988,0.030,0.972,0.015,0.989,0.000
7,DESKNN,0.973,0.015,0.959,0.002,0.988,0.030,0.973,0.015,0.990,0.001
8,FIRE-MCB,0.957,0.013,0.962,0.003,0.951,0.027,0.956,0.014,0.961,0.028
9,MCB,0.956,0.014,0.960,0.004,0.950,0.027,0.955,0.015,0.962,0.028


In [22]:
finalResultsDF.to_csv("Results/mix_pool_results/des_mix_pool_9_cls_results_5-4.csv", index=False)

In [19]:
classifiers_names = ["MLP", "Decision Tree", "Logistic Regression", "SVC", "NB", "Random Forest", "CatBoost",
                     "LGBM", "MajorityVoting"] 
# "AdaBoost", "LGBM"

accuracy_list = []
accuracy_std_list = []
for n in classifiers_names:
    accuracy_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.mean())
    accuracy_std_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.std())

final_cls_results = {"classifier": classifiers_names, 
                     "accuracy": accuracy_list, 
                     "accuracy_std": accuracy_std_list}

finalClassifierResultsDF = pd.DataFrame.from_dict(final_cls_results)

In [20]:
finalClassifierResultsDF

,classifier,accuracy,accuracy_std
0,MLP,0.944948,1.191249e-02
1,Decision Tree,0.930636,2.735127e-04
2,Logistic Regression,0.941748,4.596760e-16
3,SVC,0.932578,0.000000e+00
4,NB,0.904531,4.596760e-16
5,Random Forest,0.933981,1.447552e-02
6,CatBoost,0.944193,1.046654e-02
7,LGBM,0.932039,3.447570e-16
8,MajorityVoting,0.930636,2.735127e-04


In [21]:
finalResultsDF.describe().T

,count,mean,std,min,25%,50%,75%,max
accuracy,14.0,0.957857,0.015713,0.927,0.94550,0.9580,0.97275,0.974
accuracy_std,14.0,0.009143,0.008646,0.001,0.00100,0.0075,0.01500,0.025
precision,14.0,0.960643,0.009044,0.948,0.95225,0.9600,0.96875,0.973
precision_std,14.0,0.002357,0.000633,0.002,0.00200,0.0020,0.00275,0.004
recall,14.0,0.955357,0.041086,0.879,0.91725,0.9505,0.99700,1.000
recall_std,14.0,0.017143,0.018773,0.000,0.00000,0.0135,0.03000,0.051
f1_score,14.0,0.957214,0.017039,0.923,0.94425,0.9570,0.97300,0.975
f1_std,14.0,0.009429,0.009246,0.001,0.00100,0.0075,0.01500,0.026
auc,14.0,0.966643,0.031265,0.909,0.96125,0.9780,0.98975,0.992
auc_std,14.0,0.006714,0.010440,0.000,0.00000,0.0005,0.00950,0.028
